## DCF_1 ###

- Basic version of DCF 
- Acquire data from Yahoo finance using "yfinance"


step :
- get > Average Free cash flow , shares , Net Debt = Long Term Borrowings - Cash
- input > growth rate , terminal growth rate , discount rate , horizons
- calculate > FCF 
- calculate > terminal value
- calculate > present value
- calculate > Intrinsic value
- calculate > Margin of safety
- summary

In [2]:
import pandas as pd
import yfinance as yf

In [3]:
ticker = 'mega.bk'
data = yf.Ticker(ticker)
data.info

{'address1': '909, Ample Tower',
 'address2': '9th floor Debaratna Road Bangna Nuea, Bangna',
 'city': 'Bangkok',
 'zip': '10260',
 'country': 'Thailand',
 'phone': '66 2 769 4222',
 'fax': '66 2 769 4244',
 'website': 'https://www.megawecare.com',
 'industry': 'Drug Manufacturers - Specialty & Generic',
 'industryKey': 'drug-manufacturers-specialty-generic',
 'industryDisp': 'Drug Manufacturers - Specialty & Generic',
 'sector': 'Healthcare',
 'sectorKey': 'healthcare',
 'sectorDisp': 'Healthcare',
 'longBusinessSummary': "Mega Lifesciences Public Company Limited, together with its subsidiaries, manufactures and sells health food supplements, prescription pharmaceutical products, over-the-counter products, herbal products, vitamins, and fast-moving consumer goods in Southeast Asia and Sub-Saharan Africa. The company operates in three segments: Brands, Distribution, and Original Equipment Manufacture (OEM). It provides various medicines for allergies; blood circulation and memory; bone

In [21]:
### - get > Average Free cash flow , shares , Net Debt = Long Term Borrowings - Cash ###

## Shares ## 
shares = data.info['sharesOutstanding']
print('Ticker = ',ticker,' Shares = ',shares)

## Net Debt = Long Term Borrowings - Cash ###
totalnoncurrentliabilities = data.balancesheet.loc[data.balancesheet.index == 'Total Non Current Liabilities Net Minority Interest'].values[0][0] - data.balancesheet.loc[data.balancesheet.index == 'Cash And Cash Equivalents'].values[0][0]
print('Ticker = ',ticker,' Net Debt = ',totalnoncurrentliabilities)

## Average Free Cash Flow ## 


Ticker =  mega.bk  Shares =  871870016
Ticker =  mega.bk  Net Debt =  -1980612000.0
